# seqGAN
ここでは、make_datasetで準備した学習データを用いて、seqGANによるキャッチコピーの生成を行う

google colabにて実行


参考サイト（https://qiita.com/everylittle/items/19c4988a135d36150dc0）

こちらの方のコードにほぼ乗っかる形で、ところどころで自分のデータに合わせながら作成しました。

とてもお世話になりました。


# seqGANのコード一覧

generator.py ：生成器について

discriminator.py：識別器について

dataloader.py：入力データを読み込んでリストを作る

rnnlm.py：生成器で使用するlstmの雛形を作ってる感じ、ワンバッチとか

rollout.py：生成器と識別器のパラメータを調整している

target_lstm.py：lstmのモデルを作っている

utils.py：ボキャブラリーの作成、生成器の事前学習、識別器と生成器の掛け合いについて、正直一番よくわかっていない部分

sequence_gan.py：メイン関数がある、全体の統括、ここでパラメータをいじればいいって書いてあった

# それぞれのコードの中身について調べてみる

# dataloader

データを読み込む部分

今回は、観光地レビューの575文字列をid化したものを読み込ませる

データセットは生成器用と識別器用のふたつを作る

識別器のデータセットでは正例と負例を区別するためにラベル付けが行われる

In [ ]:
#ライブラリのインポート
import tensorflow as tf
import numpy as np

In [ ]:
#生成器のデータセットを作成する

def dataset_for_generator(data_file, batch_size):
    dataset = []
    with open(data_file, 'r') as f:
        # 一行ずつ読み取る
        for line in f:
            line = line.strip()
            line = line.split()
            parse_line = [int(x) for x in line]
            if len(parse_line) == 15: # 一行で15単語
                dataset.append(parse_line)
    output = tf.data.Dataset.from_tensor_slices(dataset).shuffle(len(dataset)).batch(batch_size)
    return output

バッチサイズの仕組み

tf.data.Dataset.batch(10)ってやると、データをバッチサイズごとに分割してくれる

例えば、100個分のデータがあったとして、そのままだと、二次元配列に一つだけのデータ
(１００個)が入った状態になる。

[[１００個のデータ]]

これを.batch(10)ってすると、

[[１０個のデータ]]

[[１０個のデータ]]

...

[[１０個のデータ]]

みたいに、100/10　個に分けてデータを扱うことができるようになる。　


In [ ]:
#識別器のデータセットを作成

def dataset_for_discriminator(positive_file, negative_file, batch_size):

    examples = [] # データを入れる配列
    labels = [] # データがtrueかfalseかのラベルを付ける

    with open(positive_file) as fin:
        for line in fin:
            line = line.strip()
            line = line.split()
            parse_line = [int(x) for x in line]
            if len(parse_line) == 15:
                examples.append(parse_line)
                labels.append([0, 1])

    with open(negative_file) as fin:
        for line in fin:
            line = line.strip()
            line = line.split()
            parse_line = [int(x) for x in line]
            if len(parse_line) == 15:
                examples.append(parse_line)
                labels.append([1, 0])
    output = tf.data.Dataset.from_tensor_slices((examples, labels)).shuffle(len(examples)).batch(batch_size).repeat(6)
    return output

# tf.data.Datasetの形式は配列ではないのでそのままでは内容が確認できない
# for文とかnumpy()を使えば確認できるらしい
# もしくはデータの形式を変えたら確認できるのかな

# listの形式でいけたわ

# rnnlm

生成器のモデルを作成している

のちのtarget_lstmではここで作ったクラスを継承している

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, LSTM, Flatten
import numpy as np

In [ ]:
class RNNLM(object):
    def __init__(self, num_emb, batch_size, emb_dim, hidden_dim, sequence_length, start_token, learning_rate=0.01):
        self.num_emb = num_emb
        self.batch_size = batch_size
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim
        self.sequence_length = sequence_length
        self.start_token = start_token
        self.start_token_vec = tf.constant([start_token] * self.batch_size, dtype=tf.int32)
        self.learning_rate = learning_rate
        self.grad_clip = 5.0

        # 生成器のモデルを作る
        # 入力サイズ(Input)は固定の単語数
        # モデルの構造は、　input embedding LSTM Dense って流れ

        self.g_model = tf.keras.models.Sequential([
            Input((self.sequence_length,), dtype=tf.int32),
            Embedding(self.num_emb, self.emb_dim, embeddings_initializer=tf.random_normal_initializer(stddev=0.1)),
            LSTM(self.hidden_dim, kernel_initializer=tf.random_normal_initializer(stddev=0.1), recurrent_initializer=tf.random_normal_initializer(stddev=0.1), return_sequences=True),
            Dense(self.num_emb, kernel_initializer=tf.random_normal_initializer(stddev=0.1), activation="softmax")
        ])

        # optimizer(最適化関数)を自分で作ってる
        # 最適化関数　→　損失関数をどの方向にどのくらい更新するかを決める
        # learing_rate →　どの程度更新するか
        # 一回の学習で更新しすぎないように制限するパラメータも存在する
        # clipnorm →　ベクトルの長さ
        # clipvalue →　絶対値

        # 損失関数　→　sparse_categorical_crossentropy
        # これは、クラス分類に使われる損失関数
        # categorical_crossentropyだと、one-hotベクトルでの分類を行うが、sparse_categorical_crossentropyでは整数で分類可能
        # つまり[001][010][100]って分類か[0][1][2]って分類か
        # lstmだと次に来る単語の候補の確率から、どの候補かを分類してるって感じなのかな
        

        self.g_optimizer = self._create_optimizer(learning_rate, clipnorm=self.grad_clip)
        if self.g_optimizer is not None:
            self.g_model.compile(
                optimizer=self.g_optimizer,
                loss="sparse_categorical_crossentropy")
        else:
            self.g_model.compile(
                loss="sparse_categorical_crossentropy")
        self.g_embeddings = self.g_model.trainable_weights[0]


    # データセットを入れて、損失を出力する
    def target_loss(self, dataset):
        # dataset: each element has [self.batch_size, self.sequence_length]
        # outputs are 1 timestep ahead
        ds = dataset.map(lambda x: (tf.pad(x[:, 0:-1], ([0, 0], [1, 0]), "CONSTANT", self.start_token), x))
        loss = self.g_model.evaluate(ds, verbose=1)

        # print("\n")
        # print("生成したデータセットから損失を計算しました")
        return loss


    @tf.function # これを付けると、これ以降は処理速度が早くなるらしい

    # バッチサイズひとつ分だけを生成する場合
    def generate_one_batch(self):
        # パラメータの初期設定
        # h0とc0は隠れ層でのパラメータ
        # gen_xは
        h0 = c0 = tf.zeros([self.batch_size, self.hidden_dim])
        gen_x = tf.TensorArray(dtype=tf.int32, size=self.sequence_length,
                               dynamic_size=False, infer_shape=True)

        def _g_recurrence(i, x_t, h_tm1, gen_x):
            # o_t: batch x vocab, probability
            # h_t: hidden_memory_tuple
            o_t, h_t = self.g_model.layers[1].cell(x_t, h_tm1, training=False) # layers[1]: LSTM
            o_t = self.g_model.layers[2](o_t) # layers[2]: Dense
            log_prob = tf.math.log(o_t)
            next_token = tf.cast(tf.reshape(tf.random.categorical(log_prob, 1), [self.batch_size]), tf.int32)
            x_tp1 = tf.nn.embedding_lookup(self.g_embeddings, next_token)  # batch x emb_dim
            gen_x = gen_x.write(i, next_token)  # indices, batch_size
            return i + 1, x_tp1, h_t, gen_x

        # while_loop
        # cond →　ループの条件
        # body →　状態の更新
        # loop_vars →　初期状態
        _, _, _, gen_x = tf.while_loop(
            cond=lambda i, _1, _2, _3: i < self.sequence_length,
            body=_g_recurrence,
            loop_vars=(tf.constant(0, dtype=tf.int32),
                       tf.nn.embedding_lookup(self.g_embeddings, self.start_token_vec), [h0, c0], gen_x))

        gen_x = gen_x.stack()  # seq_length x batch_size
        outputs = tf.transpose(gen_x, perm=[1, 0])  # batch_size x seq_length
        return outputs

    # 生成を実行
    def generate_samples(self, num_batches, output_file):
        # Generate Samples
        with open(output_file, 'w') as fout:
            for _ in range(num_batches):
                generated_samples = self.generate_one_batch().numpy()

                # print("\n")
                # print("生成結果 : ",generated_samples) # 生成結果を表示

                # generated_samplesはバッチサイズ個作られているので、それぞれをfor文で回して出力する
                for poem in generated_samples:
                    print(' '.join([str(x) for x in poem]), file=fout)

    # __init__で使われてる
    def _create_optimizer(self, *args, **kwargs):
        return None


# target_lstm

lstmの層のサイズを設定している

元の論文にてtarget_params.pklという既に調整されたパラメータのファイルを読み込む形になっていたが、今回は使用しないため、入力サイズを自分で調整する必要がある

In [ ]:
import numpy

In [ ]:
# lstmのパラメータが入っているpickleファイルを読み取る時に使う　→　今回は使わない
# モデルのサイズはパラメータファイルで決められるよ
# パラメータファイルと自分のデータのファイルで違うので自分のファイルに合わせるように変数を調整する

class TARGET_LSTM(RNNLM):
    # function for reading save/target_params.pkl
    
    # voc_sizeを追加 num_embの値をvoc_sizeに変更
    def __init__(self, batch_size, sequence_length, start_token, params, voc_size):

#         num_emb = params[0].shape[0]
        num_emb = voc_size
        emb_dim = params[0].shape[1]
        hidden_dim = params[1].shape[1]
        
        param_0 = np.zeros((num_emb, emb_dim)) # 作成
        param_13 = np.zeros((emb_dim, num_emb)) # 作成
        param_14 = np.zeros(num_emb) # 作成

        super(TARGET_LSTM, self).__init__(num_emb, batch_size, emb_dim, hidden_dim, sequence_length, start_token)
        weights = [
            # Embedding
            param_0,
            # LSTM
            np.c_[params[1], params[4], params[10], params[7]], # kernel (i, f, c, o)
            np.c_[params[2], params[5], params[11], params[8]], # recurrent_kernel
            np.r_[params[3], params[6], params[12], params[9]], # bias
            # Dense
            param_13,
            param_14
        ]
        self.g_model.set_weights(weights)

# generator

seqGANでは、生成器は事前学習と敵対学習の２つのステップがある。

まず、事前学習にてある程度のキャッチコピーが作れるようになった上で、敵対学習を実施する。

どの程度の事前学習を行うかは、後のsequence_ganにて調整する

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, LSTM
import numpy as np

In [ ]:
# さっきのtarget_lstmと今回のgeneratorはrnnlmで構築したモデルを利用している
# ので、わからない関数があったときはrnnlmを見返すと良い
# 関数の中に別の関数があってその中にまた別の関数があるので、理解がちょっと難しい


class Generator(RNNLM):
    def __init__(self, num_emb, batch_size, emb_dim, hidden_dim, sequence_length, start_token, learning_rate=0.01):
        super(Generator, self).__init__(num_emb, batch_size, emb_dim, hidden_dim, sequence_length, start_token, learning_rate)

        # prepare model for GAN training
        # rnnlmでもしたけど、またコンパイルをしている　→　別物じゃん(g_modelとg_model_temporal)
        # sample_weight_modeってのが増えたのかな

        # sample_weight →　入力サンプルと同じ長さのnumpy配列で、訓練のサンプルに対する重みを格納する
        # これは損失関数をスケーリングするために、訓練中だけ使用する
        # あるいは系列データの場合において，2次元配列の(samples, sequence_length)という形式で， 
        # すべてのサンプルの各時間において異なる重みを適用できます． 
        # この場合，compile()の中でsample_weight_mode="temporal"と確実に明記すべき

        self.g_model_temporal = tf.keras.models.Sequential(self.g_model.layers)
        self.g_optimizer_temporal = self._create_optimizer(
            learning_rate, clipnorm=self.grad_clip)
        self.g_model_temporal.compile(
            optimizer=self.g_optimizer_temporal,
            loss="sparse_categorical_crossentropy",
            sample_weight_mode="temporal")

    # 事前学習にて実行

    def pretrain(self, dataset, target_lstm, num_epochs, num_steps, eval_file):
        # dataset: each element has [self.batch_size, self.sequence_length]
        # outputs are 1 timestep ahead

        # 5エポックごとに損失を求めている
        def pretrain_callback(epoch, logs):
            if epoch % 5 == 0:
                self.generate_samples(num_steps, eval_file) # ここで生成
                likelihood_dataset = dataset_for_generator(eval_file, self.batch_size) # データセットはここで作成

                # print("損失に用いたデータセットを表示")  
                # for i in likelihood_dataset:
                #     print(i.numpy())

                test_loss = target_lstm.target_loss(likelihood_dataset) # 作成したデータセットの損失を求める


                # print('pre-train epoch ', epoch, 'test_loss ', test_loss) # 損失を表示する
                buffer = 'epoch:\t'+ str(epoch) + '\tnll:\t' + str(test_loss) + '\n'
                log.write(buffer)

        # データセットを作成
        # データの先頭にstart_tokenの数字を付ける処理を行なっている
        # start_tokenが0だったため、paddingの0と被っていたのが問題かも →　start_tokenの値を変える or paddingの値を変える
        ds = dataset.map(lambda x: (tf.pad(x[:, 0:-1], ([0, 0], [1, 0]), "CONSTANT", self.start_token), x)).repeat(num_epochs)

        # 生成器のモデルの学習を実施
        # callbacks →　訓練中のモデルの状態を可視化するために使われる
        # model.fitの中で使われるよ
        # tf.keras.callbacks.LambdaCallback →　シンプルな自作コールバックを作れる
        # 今回のon_epoch_endではepochとlogの二つの位置引数が必要(他にも色々ある模様)



        # pretrain_loss = self.g_model.fit(ds, verbose=1, epochs=num_epochs, steps_per_epoch=num_steps,
        #                                  callbacks=[tf.keras.callbacks.LambdaCallback(on_epoch_end=pretrain_callback)])
        
        pretrain_loss = self.g_model.fit(ds, verbose=1, epochs=num_epochs, steps_per_epoch=num_steps) # callbackなしでちょっと様子を見る
        return pretrain_loss

    def train_step(self, x, rewards):
        # x: [self.batch_size, self.sequence_length]
        # rewards: [self.batch_size, self.sequence_length] (sample_weight)
        # outputs are 1 timestep ahead

        # __init__でコンパイルしてたモデル
        # train_on_batch　→　サンプル中の一つのバッチで勾配を更新する
        # やってることは.fit()と.train_on_batch()で同じなのかも(違いは、使うバッチが一つか複数か)
        # sample_weight →　サンプルの重み、numpy配列

        train_loss = self.g_model_temporal.train_on_batch(
            np.pad(x[:, 0:-1], ([0, 0], [1, 0]), "constant", constant_values=self.start_token), x,
            # sparse_categorical_crossentropy returns mean loss
            # here we multiply (batch_size * sequence_length) to use weighted "sum"
            sample_weight=rewards * self.batch_size * self.sequence_length)
        return train_loss

    def _create_optimizer(self, *args, **kwargs):
        return tf.keras.optimizers.Adam(*args, **kwargs)

    def save(self, filename):
        self.g_model.save_weights(filename, save_format="h5")

    def load(self, filename):
        self.g_model.load_weights(filename)


実行時に表示されるETA は estimated time of arrival の略で、1エポックあたりのトレーニングにかかる時間の予測のこと。

エポック内の処理の進捗と残りのデータ量を使ってkerasが自動で予測して出力する。

# discriminator

識別器も生成器と同様に、事前学習と敵対学習の２ステップを行う。

また、今回は生成器と識別器で扱うデータの種類が異なっている

- 生成器は事前学習にて、全ての観光地の575が与えられる　⇨　語彙や表現の幅を増やす
- 識別器は正解データ（正例）に対象の観光地の575が与えられる　⇨　生成器は識別器に与えられた観光地の575に似せるように生成を行うため、目的の観光地にあったキャッチコピーが生成される

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Input, Dense, Embedding, Conv1D, MaxPool1D, Concatenate, Flatten, Dropout

In [ ]:
# どこで使っているのか

class Highway(Model):
    def __init__(self, **kwargs):
        super(Highway, self).__init__(**kwargs)

    def build(self, input_shape):
        output_dim = input_shape[-1]
        self.dense_g = Dense(output_dim, activation="relu")
        self.dense_t = Dense(output_dim, activation="sigmoid")

    def call(self, input_tensor, training=False):
        g = self.dense_g(input_tensor, training=training)
        t = self.dense_t(input_tensor, training=training)
        o = t * g + (1. - t) * input_tensor
        return o

In [ ]:
class Discriminator(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """

    def __init__(
            self, sequence_length, num_classes, vocab_size,
            embedding_size, filter_sizes, num_filters, dropout_keep_prob, l2_reg_lambda=0.2):
      
        self.sequence_length = sequence_length
        self.num_classes = num_classes
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size

        layer_input = Input((sequence_length,), dtype=tf.int32)
        layer_emb = Embedding(vocab_size, embedding_size, embeddings_initializer=tf.random_uniform_initializer(-1.0, 1.0))(layer_input)
        # (None, sequence_length, embedding_size)


        # filter_sizes →　畳み込みのフィルターの大きさ(カーネルサイズのこと)
        # num_filters →　フィルターの数(畳み込みにおける出力フィルタの数)
        pooled_outputs = []
        for filter_size, num_filter in zip(filter_sizes, num_filters):
            x = Conv1D(num_filter, filter_size)(layer_emb) # (None, sequence_length - filter_size + 1, num_filter)
            x = MaxPool1D(sequence_length - filter_size + 1)(x) # (None, 1, num_filter)
            pooled_outputs.append(x)

        x = Concatenate()(pooled_outputs)
        x = Flatten()(x) # (None, sum(num_filters))
        x = Highway()(x)
        x = Dropout(1.0 - dropout_keep_prob)(x)
        layer_output = Dense(num_classes,
                             kernel_regularizer=tf.keras.regularizers.l2(l2_reg_lambda),
                             bias_regularizer=tf.keras.regularizers.l2(l2_reg_lambda),
                             activation="softmax")(x)

        self.d_model = Model(layer_input, layer_output)
        d_optimizer = tf.keras.optimizers.Adam(1e-4)
        self.d_model.compile(optimizer=d_optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    def train(self, dataset, num_epochs, num_steps, **kwargs):
        # dataset: ([None, sequence_length], [None, num_classes])
        return self.d_model.fit(dataset.repeat(num_epochs), verbose=1, epochs=num_epochs, steps_per_epoch=num_steps, **kwargs)

    def save(self, filename):
        self.d_model.save_weights(filename, save_format="h5")

    def load(self, filename):
        self.d_model.load_weights(filename)

# rollout

敵対学習にてパラメータの調整を行う部分

また、生成器に対する強化学習の手法によるパラメータの更新の部分のここで設定している。

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
class ROLLOUT(RNNLM):
    def __init__(self, lstm, update_rate):
        super(ROLLOUT, self).__init__(lstm.num_emb, lstm.batch_size, lstm.emb_dim, lstm.hidden_dim, lstm.sequence_length, lstm.start_token)

        # generatorのモデルをlstmとして読み込む
        # update_rate = 0.8がデフォルトの設定
        # .get_weights()でモデルの重みを取得して、set_weightsで別のモデルの重みとして設定
        self.lstm = lstm
        self.update_rate = update_rate
        self.g_model.set_weights(lstm.g_model.get_weights())


    @tf.function
    def generate_one_batch(self, x_orig, given_num):
        # Initial states
        h0 = c0 = tf.zeros([self.batch_size, self.hidden_dim])
        h0 = [h0, c0]

        # tf.transpose →　元の行列をpermで指定した順番に変える
        # 今回はperm=[1, 0, 2] なので、index[0] →　index[1]になり、index[1] →　index[0]になる
        processed_x = tf.transpose(tf.nn.embedding_lookup(self.g_embeddings, x_orig), perm=[1, 0, 2])  # seq_length x batch_size x emb_dim

        # tf.TensorArray →　Tensor Arrayを生成する。多分配列みたいなものだと思う
        # size →　tensorarrayのサイズ
        # dynamic_size →　sizeを動的に変更できよるようにするか
        # clear_after_read →　readでデータを取得した後に初期化するか

        # .wirte →　データを書き込む(index →　書き込む位置, value →　書き込む値)
        # .read →　要素を一つ見る
        # .stack　→　全ての要素を見る
        # .unstack →　配列を一気に書き込む
        gen_x = tf.TensorArray(dtype=tf.int32, size=self.sequence_length,
                                             dynamic_size=False, infer_shape=True)

        ta_emb_x = tf.TensorArray(dtype=tf.float32, size=self.sequence_length)
        ta_emb_x = ta_emb_x.unstack(processed_x)
        ta_x = tf.TensorArray(dtype=tf.int32, size=self.sequence_length)
        ta_x = ta_x.unstack(tf.transpose(x_orig, perm=[1, 0]))

        # When current index i < given_num, use the provided tokens as the input at each time step
        # 最初は既存のlstmのモデルからパラメータを取得しているが、given_numを越えると、パラメータを元にnext_tokenの確率を計算し直して算出している

        def _g_recurrence_1(i, x_t, h_tm1, given_num, gen_x):
            # h_t: hidden_memory_tuple
            _, h_t = self.g_model.layers[1].cell(x_t, h_tm1, training=False) # layers[1]: LSTM
            x_tp1 = ta_emb_x.read(i)
            next_token = ta_x.read(i)
            gen_x = gen_x.write(i, next_token)  # indices, batch_size
            return i + 1, x_tp1, h_t, given_num, gen_x

        # When current index i >= given_num, start roll-out, use the output as time step t as the input at time step t+1
        def _g_recurrence_2(i, x_t, h_tm1, given_num, gen_x):
            # o_t: batch x vocab, probability
            # h_t: hidden_memory_tuple
            o_t, h_t = self.g_model.layers[1].cell(x_t, h_tm1, training=False) # layers[1]: LSTM
            o_t = self.g_model.layers[2](o_t) # layers[2]: Dense
            log_prob = tf.math.log(o_t)
            next_token = tf.cast(tf.reshape(tf.random.categorical(log_prob, 1), [self.batch_size]), tf.int32)
            x_tp1 = tf.nn.embedding_lookup(self.g_embeddings, next_token)  # batch x emb_dim
            gen_x = gen_x.write(i, next_token)  # indices, batch_size
            return i + 1, x_tp1, h_t, given_num, gen_x

        # tf.while_loopでループさせる
        # i < given_numの場合と、given_num < i < sequence_lengthの場合で、異なるループを回す
        i, x_t, h_tm1, given_num, gen_x = tf.while_loop(
            cond=lambda i, _1, _2, given_num, _4: i < given_num,
            body=_g_recurrence_1,
            loop_vars=(tf.constant(0, dtype=tf.int32),
                       tf.nn.embedding_lookup(self.g_embeddings, self.start_token_vec), h0, given_num, gen_x))

        _, _, _, _, gen_x = tf.while_loop(
            cond=lambda i, _1, _2, _3, _4: i < self.sequence_length,
            body=_g_recurrence_2,
            loop_vars=(i, x_t, h_tm1, given_num, gen_x))

        # gen_x: seq_length x batch_size
        # 最終的にgen_xをoutputする
        # gen_xには、単語の順番と次の単語の確率が入っているはず
        outputs = tf.transpose(gen_x.stack(), perm=[1, 0])  # batch_size x seq_length
        return outputs



    # 一度generatorでワンバッチ分のsampleを生成し、それを入力している
    # 生成したサンプルからnext_tokenの確率をもう一度算出し直している
    # その後、算出した確率をdiscriminatorのモデルに入れてypred_for_accを算出し、rewordsに追加していく
    def get_reward(self, input_x, rollout_num, discriminator):
        rewards = []
        for i in range(rollout_num):
            # given_num between 1 to sequence_length - 1 for a part completed sentence
            for given_num in tf.range(1, self.sequence_length):
                samples = self.generate_one_batch(input_x, given_num) # 文章をバッチサイズ個だけ生成
                ypred_for_auc = discriminator.d_model(samples).numpy() # 入力した文章に対して[falseの確率 trueの確率]を返す
                ypred = ypred_for_auc[:, 1] # prob for outputting 1 (True) つまり　Trueの確率のみを取り出す
                if i == 0:
                    rewards.append(ypred)
                else:
                    rewards[given_num - 1] += ypred

            # the last token reward
            ypred_for_auc = discriminator.d_model(input_x).numpy()
            ypred = ypred_for_auc[:, 1]
            if i == 0:
                rewards.append(ypred)
            else:
                # completed sentence reward
                rewards[self.sequence_length - 1] += ypred

        # rollout_num回for文で回してから、rollout_numで割ってるってことは、平均を出してるってことかも
        rewards = np.transpose(np.array(rewards)) / (1.0 * rollout_num)  # batch_size x seq_length
        return rewards

    def update_params(self):
        # Weights and Bias for input and hidden tensor
        # self: Rollout, self.lstm: Original generator

        # The embedding layer: directly (fully) transferred
        # The other layers: transferred with the ratio of (1 - self.update_rate)

        # 生成器で構築したg_modelとrolloutで再び構築したlstm.g_modelのパラメータを掛け合わせて調節し新しい重みを算出している
        # g_modelが0.8でlstm.g_modelが0.2って感じ
        new_weights = [self.update_rate * w1 + (1 - self.update_rate) * w2 if i > 0 else w2
                       for i, (w1, w2) in enumerate(zip(self.g_model.get_weights(), self.lstm.g_model.get_weights()))]
        self.g_model.set_weights(new_weights)


# file_output

学習のモデルの重みや生成結果、識別器が算出した本物かどうかの確率などを出力する部分

ここは自作

In [ ]:
def file_output(epoch):
    temp_output_list = []

    for _ in range(100):
        temp_output = generator.generate_one_batch()

        prob = discriminator.d_model(temp_output).numpy()
        ypred = prob[:, 1]


        # numpy形式のリストに変換したのち、pythonのリストの形式に変換
        temp_output = temp_output.numpy()
        temp_output = temp_output.tolist()

        # print(temp_output)
        # print(ypred)


        for o, y in zip(temp_output, ypred):
          temp = [o,y]
          temp_output_list.append(temp)

    temp_output_list.sort(reverse=True, key=lambda x:x[1])

    # 生成結果をテキスト形式にする
    output_text = ""
    for i in temp_output_list:
      temp_text = ""
      for out in i[0]:
        temp_text += str(out)
        temp_text += " "
      
      output_text += temp_text
      output_text += "\n"

    # print(output_text)

    # 生成結果を出力
    with open('data/generated/output_text_{}.txt'.format(epoch), 'w') as f:
      f.write(output_text)


    # 生成結果の確率をテキスト形式する
    output_text = ""
    for i in temp_output_list:
      output_text += str(i[1])
      output_text += "\n"

    # print(output_text)

    # 生成結果を出力
    with open('data/generated/prob_{}.txt'.format(epoch), 'w') as f:
      f.write(output_text)

    # 生成器と識別器の重みを保存
    generator.save("data/generated/generator_{}.h5".format(epoch))
    discriminator.save("data/generated/discriminator_{}.h5".format(epoch))



# sequence_gan

実行部分

パラメータやエポック数の調整はここで行う

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numpy as np
import tensorflow as tf
#tf.disable_v2_behavior()

import os
import random
import pickle

# 変えるポイント

START_TOKEN → vocab_size以内で使用していないidの数（0はパディングと被るのでやめたほうが良い）　

SEQ_LENGTH 20 →　15 今回の入力は15単語で固定

dis_num_filtersとdis_filter_sizesの配列の最後を消去

generated_num →　入力データの数と合わせる

vocab_size →　観光地の575での語彙数より大きくなるように調整

rewards = rollout.get_reward(samples, 16 →　12, discriminator)

PRE_EPOCH_NUM と　識別器の事前学習の回数は変わりうる（結果をみながら調整が必要）




今回は博多駅を対象にしており、他の観光地にて生成を行う場合は、適当に合わせること

In [ ]:
#########################################################################################
#  生成器のパラメータ
######################################################################################
EMB_DIM = 32 # embedding dimension
HIDDEN_DIM = 32 # hidden state dimension of lstm cell
SEQ_LENGTH = 15 # sequence length
START_TOKEN = 1699
PRE_EPOCH_NUM = 301 # supervise (maximum likelihood estimation) epochs
SEED = 88
BATCH_SIZE = 64

#########################################################################################
#  識別器のパラメータ
#########################################################################################
dis_embedding_dim = 64
dis_filter_sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15]
dis_num_filters = [100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160]
dis_dropout_keep_prob = 0.75
dis_l2_reg_lambda = 0.2
dis_batch_size = 64

#########################################################################################
#  Basic Training Parameters
# GANの学習を実行していく
#########################################################################################

TOTAL_BATCH = 300 # 生成器と識別器の更新を300回する

# 学習で使用するデータ
# 最初は存在しないので、lstmで作るらしい

# 生成器の事前学習で使用
generator_file = 'data/id/all_575_id.txt' 
eval_file = 'data/generated/eval_file.txt' # 使えていない気がする

# 識別器の学習で使用
positive_file = 'data/id/hakataeki_575_id.txt' # 対象とする観光地の575のid
negative_file = 'data/generated/hakataeki/paramater/generator_sample.txt' # 生成器が作成した偽物を使用する

output_file = 'data/generated/hakataeki/output/output_file.txt'

# 生成された数
generated_num = 1024 # kgeneratorのデータ数と合わせている

In [ ]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
assert START_TOKEN == 1699

vocab_size = 1700

physical_devices = tf.config.experimental.list_physical_devices("GPU")
if len(physical_devices) > 0:
    print("GPUが使えるよ")
    # for dev in physical_devices:
    #     tf.config.experimental.set_memory_growth(dev, True)
else:
    print("GPUは使えないよ,ごめんね")

In [ ]:
# 生成器で学習
generator = Generator(vocab_size, BATCH_SIZE, EMB_DIM, HIDDEN_DIM, SEQ_LENGTH, START_TOKEN)
target_params = pickle.load(open('data/generated/target_params_py3.pkl', 'rb'))
target_lstm = TARGET_LSTM(BATCH_SIZE, SEQ_LENGTH, START_TOKEN, target_params, vocab_size) # The oracle model

# 識別器で学習
discriminator = Discriminator(sequence_length=SEQ_LENGTH, num_classes=2, vocab_size=vocab_size, embedding_size=dis_embedding_dim,
                                filter_sizes=dis_filter_sizes, num_filters=dis_num_filters, dropout_keep_prob=dis_dropout_keep_prob,
                                l2_reg_lambda=dis_l2_reg_lambda)

# First, use the oracle model to provide the positive examples, which are sampled from the oracle data distribution
# GANの学習で使用する正解データを作成する
if not os.path.exists(generator_file):
    target_lstm.generate_samples(generated_num // BATCH_SIZE, generator_file)
gen_dataset = dataset_for_generator(generator_file, BATCH_SIZE)
log = open('data/generated/experiment-log.txt', 'w')


#  事前学習での文章生成をlstmで行い、生成器の重みを保存する
if not os.path.exists("data/generated/hakataeki/paramater/generator_pretrained.h5"):
    print('Start pre-training...')
    log.write('pre-training...\n')
    generator.pretrain(gen_dataset, target_lstm, PRE_EPOCH_NUM, generated_num // BATCH_SIZE, eval_file)
    generator.save("data/generated/hakataeki/paramater/generator_pretrained.h5")
else:
    generator.load("data/generated/hakataeki/paramater/generator_pretrained.h5")

# 識別器の事前学習での重み
if not os.path.exists("data/generated/hakataeki/paramater/discriminator_pretrained.h5"):
    print('Start pre-training discriminator...')
    # 1エポックの識別器の訓練を15回繰り返す
    for _ in range(15):
        print("Dataset", _)

        # まず生成器が偽物を作成
        generator.generate_samples(generated_num // BATCH_SIZE, negative_file)

        # 偽物と本物を混ぜたデータセットを作成
        dis_dataset = dataset_for_discriminator(positive_file, negative_file, BATCH_SIZE)

        # 識別器を学習させる
        discriminator.train(dis_dataset, 1, (generated_num // BATCH_SIZE) * 2)
    discriminator.save("data/generated/hakataeki/paramater/discriminator_pretrained.h5")
else:
    discriminator.load("data/generated/hakataeki/paramater/discriminator_pretrained.h5")

rollout = ROLLOUT(generator, 0.8)


In [ ]:

print('#########################################################################')
print('Start Adversarial Training...')
log.write('adversarial training...\n')

# 学習の実行
# 今回は200回の訓練を行う

# accとlossのグラフ描画のための配列
acc_list = []
loss_list = []

for total_batch in range(TOTAL_BATCH):
    print("Generator", total_batch)
    # Train the generator for one step
    for _ in range(5):
        samples = generator.generate_one_batch()
        rewards = rollout.get_reward(samples, 12, discriminator)
        generator.train_step(samples, rewards)

    # target_lossが機能していないと思われるので実行しない
    # # Test 
    # if total_batch % 5 == 0 or total_batch == TOTAL_BATCH - 1:
    #     generator.generate_samples(generated_num // BATCH_SIZE, eval_file)
    #     likelihood_dataset = dataset_for_generator(eval_file, BATCH_SIZE)
    #     test_loss = target_lstm.target_loss(likelihood_dataset)
    #     buffer = 'epoch:\t' + str(total_batch) + '\tnll:\t' + str(test_loss) + '\n'
    #     print('total_batch: ', total_batch, 'test_loss: ', test_loss)
    #     log.write(buffer)

    # Update roll-out parameters
    rollout.update_params()

    # Train the discriminator
    print("Discriminator", total_batch)
    for _ in range(1):
        generator.generate_samples(generated_num // BATCH_SIZE, negative_file)
        dis_dataset = dataset_for_discriminator(positive_file, negative_file, BATCH_SIZE)
        history = discriminator.train(dis_dataset, 1, (generated_num // BATCH_SIZE) * 2)
        
        acc_list.append(history.history['accuracy'])
        loss_list .append(history.history['loss'])


file_output(TOTAL_BATCH)

with open('data/generated/hakataeki/paramater/acc_list.pickle', 'wb') as f:
    pickle.dump(acc_list, f)

with open('data/generated/hakataeki/paramater/loss_list.pickle', 'wb') as f:
    pickle.dump(loss_list, f)



log.close()


google colabにてフォルダをダウンロードするときに使用

In [ ]:
# ダウンロードしたいフォルダを zip 圧縮する
!zip -r /content/data/generated.zip /content/data/generated

# 圧縮した zip ファイルをダウンロードする
from google.colab import files
files.download("/content/data/generated.zip")